# Model Translation + APS Viewer 

This notebook sketches how to upload with `aps_automation_sdk`, translate to SVF/SVF2, and get a Viewer-ready URN using helpers from `aps_viewer_sdk`. Also all elements in the model will be colored.

**Prereqs**
- Env vars: `CLIENT_ID`, `CLIENT_SECRET`
- Dependencies: `aps_viewer_sdk`, `aps_automation_sdk`, `requests`
- A local file to upload via Design Automation / OSS


In [1]:
import os
import time
import uuid
import random
import tempfile
from typing import cast
from pathlib import Path

import requests
from dotenv import load_dotenv

from aps_viewer_sdk.helper import (
    get_2lo_token,
    get_metadata_viewables,
    get_all_model_properties,
)
from aps_viewer_sdk import APSViewer, ElementsInScene
from aps_automation_sdk import (
    translate_file_in_oss,
    get_translation_status,
)

load_dotenv()

True

In [2]:
CLIENT_ID = os.environ.get("CLIENT_ID")
CLIENT_SECRET = os.environ.get("CLIENT_SECRET")

if not CLIENT_ID or not CLIENT_SECRET:
    raise RuntimeError("CLIENT_ID and CLIENT_SECRET must be set in environment")

## Example: upload with `aps_automation_sdk` and translate

You need `aps_automation_sdk` installed for this cell. The snippet below uses the sample file in this repo via `Path` to avoid path issues.


In [ ]:
from aps_automation_sdk import ActivityInputParameter

client_id = os.environ.get("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")
if not client_id or not client_secret:
    raise RuntimeError(
        "CLIENT_ID and CLIENT_SECRET must be set in the environment variables."
    )

sample_path = Path("SampleStructuralModel.rvt").resolve()

print("Starting model processing workflow...")
print(f"Uploading file: {sample_path.name}")

token = get_2lo_token(client_id, client_secret)

bucket_key = uuid.uuid4().hex
object_key = f"input_{uuid.uuid4()}{sample_path.suffix}"

input_param = ActivityInputParameter(
    name="inputFile",
    localName=sample_path.name,
    verb="get",
    description="Input CAD model",
    required=True,
    is_engine_input=False,
    bucketKey=bucket_key,
    objectKey=object_key,
)

input_param.upload_file_to_oss(file_path=str(sample_path), token=token)

# Use the helper from aps_automation_sdk to translate the file
viewer_urn = translate_file_in_oss(
    token=token,
    bucket_key=bucket_key,
    output_object_key=object_key,
    max_wait_time=300,
    poll_interval=15,
    verbose=True,
)
print(f"Viewer URN: {viewer_urn}")

Starting model processing workflow...
Uploading file: SampleStructuralModel.rvt
🎯 STARTING MODEL TRANSLATION FOR VIEWING
Bucket: 64b013246d174706880983bd5259c654
Object: input_28bf17c9-9cad-42f9-9065-2bdc4c99b3dc.rvt
🔗 Object URN: dXJuOmFkc2sub2JqZWN0czpvcy5vYmplY3Q6NjRiMDEzMjQ2ZDE3NDcwNjg4MDk4M2JkNTI1OWM2NTQvaW5wdXRfMjhiZjE3YzktOWNhZC00MmY5LTkwNjUtMmJkYzRjOTliM2RjLnJ2dA
🔄 Starting Model Derivative translation job...
📋 Job Details: dXJuOmFkc2sub2JqZWN0czpvcy5vYmplY3Q6NjRiMDEzMjQ2ZDE3NDcwNjg4MDk4M2JkNTI1OWM2NTQvaW5wdXRfMjhiZjE3YzktOWNhZC00MmY5LTkwNjUtMmJkYzRjOTliM2RjLnJ2dA
⏳ Monitoring translation progress...
  > MD Status: pending (0% complete)
  > MD Status: inprogress (0% complete)
  > MD Status: inprogress (0% complete)
  > MD Status: inprogress (21% complete)
  > MD Status: inprogress (48% complete)
  > MD Status: inprogress (68% complete)
  > MD Status: inprogress (99% complete)
  > MD Status: inprogress (99% complete)
  > MD Status: success (complete)
✅ Translation completed succ

## View the model with highlighted elements

Now that the model is translated, we can use `aps_viewer_sdk` to create an interactive viewer with randomly colored elements.

In [4]:
# Create the viewer with the translated model
urn_bs64 = viewer_urn

viewer = APSViewer(
    urn=f"urn:adsk.objects:os.object:{bucket_key}/{object_key}",
    token=token,
    views_selector=True,
)

# Get viewables and select the first 3D view
viewables = viewer.get_viewables(urn_bs64)
if not viewables:
    raise RuntimeError("No viewables returned for the translated model")

first_view = next((v for v in viewables if v.get("role") == "3d"), None)
if not first_view:
    raise RuntimeError("No 3D viewables returned for the translated model")

viewer.set_view_guid(first_view["guid"], first_view["name"], first_view["role"])
print(f"Selected view: {first_view['name']} (GUID: {first_view['guid']})")

# Get metadata viewables to retrieve element properties
metadata_views = get_metadata_viewables(token, urn_bs64)
if not metadata_views:
    raise RuntimeError("No metadata viewables returned for the translated model")

model_guid = next(
    (v["guid"] for v in metadata_views if v.get("role") == "3d"),
    metadata_views[0].get("guid"),
)
if not model_guid:
    raise RuntimeError("No valid model GUID available for metadata properties")

# Get all model properties to extract external IDs
payload: dict[str, object] = get_all_model_properties(token, urn_bs64, model_guid)
data_raw = payload.get("data")
data: dict[str, object] = cast(
    dict[str, object], data_raw if isinstance(data_raw, dict) else payload
)
collection = data.get("collection", [])

# Extract unique external IDs
seen: set[str] = set()
external_ids: list[str] = []
for item in collection:
    ext = item.get("externalId")
    if isinstance(ext, str) and ext and ext not in seen:
        seen.add(ext)
        external_ids.append(ext)

if not external_ids:
    raise RuntimeError("No external IDs returned for selected view")

print(f"Found {len(external_ids)} unique external IDs")
print("Sample external_ids:", external_ids[:10])

# Highlight all elements with random colors
rng = random.Random(0)
highlight: list[ElementsInScene] = []
for ext_id in external_ids:  # Highlight all elements
    color = "#{:02x}{:02x}{:02x}".format(
        rng.randrange(256), rng.randrange(256), rng.randrange(256)
    )
    highlight.append({"externalElementId": ext_id, "color": color})

viewer.highlight_elements(highlight)
print(f"Highlighted {len(highlight)} elements with random colors")

# Display the viewer
viewer.show()

Selected view: Structure Only (GUID: b8b1444c-7cc1-0bc9-a5c4-a4cd8d15c564)
Found 1343 unique external IDs
Sample external_ids: ['42a41bc2-12e7-489c-96cb-c2f1ff6449f1-00000149', '33e279c0-7506-4b11-8b15-bba7cca495fb-00007f37', '9e748b35-2e51-4170-b964-a9c0f3c563d9-0008d1b8', '9e748b35-2e51-4170-b964-a9c0f3c563d9-0008d203', '9e748b35-2e51-4170-b964-a9c0f3c563d9-0008d20a', '9e748b35-2e51-4170-b964-a9c0f3c563d9-0008d213', 'ef74f80b-36f6-466f-931f-2f08bd3355e9-0008dd9e', '63f8d234-f38c-4508-90c8-59b76e38f488-0008eff2', 'be230f29-8239-4c14-8dea-73ab8498a467-00095e4f', 'e86e0fb0-aaef-4c8d-af36-9f1e0c4f7962-00096e54']
Highlighted 1343 elements with random colors


## Result

Here's what the viewer looks like with highlighted elements:

![Highlighted Elements Result](../../assets/example1.png)

Each element in the model is colored with a random color, making it easy to distinguish individual components. This technique is useful for:
- Quality assurance and visual inspection
- Element identification and selection
- Creating color-coded visualizations based on properties
- Interactive model exploration